# 🏥 RoboCare AI Assistant

## Why I Built This

While working on a caregiver matching platform for **MyWoosah Inc** in the US, I faced a real challenge: how do you efficiently match families with the right caregivers when everyone has different needs?

Families would ask things like:
- *"I need someone for my mom on Monday mornings who speaks Spanish"*
- *"Can you find elder care in Boston under $30/hour with CPR certification?"*

Writing individual SQL queries for every combination of filters was exhausting and error-prone. I knew there had to be a better way.

That's when I discovered the **Andela LLM Engineering program**. I saw an opportunity to transform this problem into a solution using AI. Instead of rigid queries, what if families could just... talk? And the AI would understand, search, and recommend?

This project is my answer. It's not just an exercise—it's solving a real problem I encountered in the field.

---

## What This Does

RoboCare helps families find caregivers through natural conversation. You tell it what you need, and it:
- 🔍 Searches the database intelligently
- 🎯 Finds the best matches
- 💬 Explains pros/cons in plain English  
- 🔊 Speaks the results back to you

**Tech:** OpenAI GPT-4o + Voice • Gradio UI • SQLite Database • Function Calling

---

**Note:** This is a demonstration. Always verify credentials independently.

## Step 1: Libraries

The essentials: OpenAI for the AI brain, Gradio for the interface, SQLite for data storage.


In [63]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3
import sqlite3
from textwrap import dedent
from contextlib import contextmanager
from typing import Optional, List, Dict, Any, Tuple

## Step 2: Setup

Loading API keys securely (never hardcode them!), setting up the OpenAI client, and pointing to our database.


In [64]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

DB_PATH = "care_app.db"

OpenAI API Key exists and begins sk-proj-


## Step 3: The Database

20 sample caregivers across major US cities with:
- Services they offer (elder care, child care, etc.)
- Languages, certifications, availability
- Personality traits
- Realistic pricing and schedules

This mirrors the kind of data MyWoosah Inc would manage—except here, AI does the matching work.


In [65]:
# Table creation and seeding

SQL = '''

  CREATE TABLE IF NOT EXISTS caregivers (
    id               INTEGER PRIMARY KEY,
    name             TEXT NOT NULL,
    gender           TEXT,
    years_experience INTEGER,
    live_in          INTEGER,      -- 0/1
    hourly_rate      REAL,
    currency         TEXT,
    city             TEXT,
    state_province   TEXT,
    country          TEXT,
    postal_code      TEXT,
    lat              REAL,
    lon              REAL
  );

  CREATE TABLE IF NOT EXISTS caregiver_services (
    caregiver_id INTEGER,
    care_type    TEXT,
    FOREIGN KEY (caregiver_id) REFERENCES caregivers(id)
  );

  CREATE TABLE IF NOT EXISTS availability (
    caregiver_id INTEGER,
    day          TEXT,     -- e.g., 'Mon'
    time_start   TEXT,     -- 'HH:MM'
    time_end     TEXT,     -- 'HH:MM'
    FOREIGN KEY (caregiver_id) REFERENCES caregivers(id)
  );

  CREATE TABLE IF NOT EXISTS languages (
    caregiver_id INTEGER,
    language     TEXT,
    FOREIGN KEY (caregiver_id) REFERENCES caregivers(id)
  );

  CREATE TABLE IF NOT EXISTS certifications (
    caregiver_id INTEGER,
    cert         TEXT,
    FOREIGN KEY (caregiver_id) REFERENCES caregivers(id)
  );

  CREATE TABLE IF NOT EXISTS traits (
    caregiver_id INTEGER,
    trait        TEXT,
    FOREIGN KEY (caregiver_id) REFERENCES caregivers(id)
  );

  ----------------------------------------------------------

  -- Clear old data (optional)

  DELETE FROM traits;
  DELETE FROM certifications;
  DELETE FROM languages;
  DELETE FROM availability;
  DELETE FROM caregiver_services;
  DELETE FROM caregivers;

  -- Seed caregivers (20 examples, all USA)

  INSERT INTO caregivers
  (id, name, gender, years_experience, live_in, hourly_rate, currency, city, state_province, country, postal_code, lat, lon)
  VALUES
  (1,  'Grace Williams',     'female', 6,  0, 28, 'USD', 'New York',       'NY', 'USA', '10001',  40.7128,  -74.0060),
  (2,  'Miguel Alvarez',     'male',   9,  1, 30, 'USD', 'Los Angeles',    'CA', 'USA', '90012',  34.0522, -118.2437),
  (3,  'Ava Johnson',        'female', 4,  0, 24, 'USD', 'Chicago',        'IL', 'USA', '60601',  41.8781,  -87.6298),
  (4,  'Noah Robinson',      'male',   12, 0, 27, 'USD', 'Houston',        'TX', 'USA', '77002',  29.7604,  -95.3698),
  (5,  'Sophia Martinez',    'female', 8,  0, 29, 'USD', 'Phoenix',        'AZ', 'USA', '85004',  33.4484, -112.0740),
  (6,  'Daniel Carter',      'male',   10, 1, 31, 'USD', 'Philadelphia',   'PA', 'USA', '19103',  39.9526,  -75.1652),
  (7,  'Emily Nguyen',       'female', 7,  0, 26, 'USD', 'San Antonio',    'TX', 'USA', '78205',  29.4241,  -98.4936),
  (8,  'Olivia Kim',         'female', 5,  0, 27, 'USD', 'San Diego',      'CA', 'USA', '92101',  32.7157, -117.1611),
  (9,  'James Thompson',     'male',   15, 1, 34, 'USD', 'Dallas',         'TX', 'USA', '75201',  32.7767,  -96.7970),
  (10, 'Isabella Garcia',    'female', 3,  0, 22, 'USD', 'San Jose',       'CA', 'USA', '95113',  37.3382, -121.8863),
  (11, 'Ethan Patel',        'male',   11, 1, 33, 'USD', 'Austin',         'TX', 'USA', '78701',  30.2672,  -97.7431),
  (12, 'Harper Brooks',      'female', 2,  0, 20, 'USD', 'Jacksonville',   'FL', 'USA', '32202',  30.3322,  -81.6557),
  (13, 'Logan White',        'male',   6,  0, 25, 'USD', 'Fort Worth',     'TX', 'USA', '76102',  32.7555,  -97.3308),
  (14, 'Amelia Davis',       'female', 9,  0, 28, 'USD', 'Columbus',       'OH', 'USA', '43215',  39.9612,  -82.9988),
  (15, 'Charlotte Reed',     'female', 14, 1, 32, 'USD', 'Charlotte',      'NC', 'USA', '28202',  35.2271,  -80.8431),
  (16, 'Jackson Lee',        'male',   5,  0, 26, 'USD', 'San Francisco',  'CA', 'USA', '94102',  37.7749, -122.4194),
  (17, 'Avery Chen',         'female', 7,  0, 27, 'USD', 'Seattle',        'WA', 'USA', '98101',  47.6062, -122.3321),
  (18, 'William Turner',     'male',   13, 1, 35, 'USD', 'Denver',         'CO', 'USA', '80202',  39.7392, -104.9903),
  (19, 'Natalie O''Brien',   'female', 16, 0, 36, 'USD', 'Boston',         'MA', 'USA', '02108',  42.3601,  -71.0589),
  (20, 'Maya Robinson',      'female', 3,  0, 23, 'USD', 'Atlanta',        'GA', 'USA', '30303',  33.7488,  -84.3880);

  -- Seed caregiver services

  INSERT INTO caregiver_services (caregiver_id, care_type) VALUES
  (1,  'elder care'),           (1,  'companionship'),
  (2,  'post-op support'),      (2,  'elder care'),
  (3,  'child care'),           (3,  'special needs'),
  (4,  'respite care'),         (4,  'elder care'),
  (5,  'dementia care'),        (5,  'companionship'),
  (6,  'elder care'),           (6,  'hospice support'),
  (7,  'child care'),           (7,  'respite care'),
  (8,  'post-op support'),      (8,  'companionship'),
  (9,  'special needs'),        (9,  'elder care'),
  (10, 'child care'),           (10, 'companionship'),
  (11, 'dementia care'),        (11, 'post-op support'),
  (12, 'child care'),           (12, 'special needs'),
  (13, 'respite care'),         (13, 'companionship'),
  (14, 'elder care'),           (14, 'post-op support'),
  (15, 'hospice support'),      (15, 'dementia care'),
  (16, 'elder care'),           (16, 'respite care'),
  (17, 'special needs'),        (17, 'companionship'),
  (18, 'post-op support'),      (18, 'elder care'),
  (19, 'dementia care'),        (19, 'hospice support'),
  (20, 'child care'),           (20, 'companionship');

  -- Seed availability (Mon-Sun samples)

  INSERT INTO availability (caregiver_id, day, time_start, time_end) VALUES
  -- 1 Grace (NY): evenings + Sun
  (1, 'Mon', '17:30', '22:00'),
  (1, 'Thu', '17:30', '22:00'),
  (1, 'Sun', '10:00', '16:00'),
  -- 2 Miguel (LA): live-in, long blocks
  (2, 'Tue', '08:00', '20:00'),
  (2, 'Thu', '08:00', '20:00'),
  (2, 'Sat', '09:00', '18:00'),
  -- 3 Ava (CHI): weekdays 09-17
  (3, 'Mon', '09:00', '17:00'),
  (3, 'Wed', '09:00', '17:00'),
  (3, 'Fri', '09:00', '17:00'),
  -- 4 Noah (HOU): Tue-Fri 08-16
  (4, 'Tue', '08:00', '16:00'),
  (4, 'Wed', '08:00', '16:00'),
  (4, 'Thu', '08:00', '16:00'),
  -- 5 Sophia (PHX): Thu-Sun 10-18
  (5, 'Thu', '10:00', '18:00'),
  (5, 'Fri', '10:00', '18:00'),
  (5, 'Sat', '10:00', '18:00'),
  -- 6 Daniel (PHL): Mon-Thu 07-15
  (6, 'Mon', '07:00', '15:00'),
  (6, 'Tue', '07:00', '15:00'),
  (6, 'Thu', '07:00', '15:00'),
  -- 7 Emily (SAT): weekends
  (7, 'Sat', '08:00', '17:00'),
  (7, 'Sun', '09:00', '17:00'),
  (7, 'Fri', '17:00', '21:00'),
  -- 8 Olivia (SD): Mon, Wed evenings
  (8, 'Mon', '16:00', '21:00'),
  (8, 'Wed', '16:00', '21:00'),
  (8, 'Sat', '10:00', '14:00'),
  -- 9 James (DAL): live-in wide
  (9, 'Mon', '07:00', '19:00'),
  (9, 'Wed', '07:00', '19:00'),
  (9, 'Sun', '09:00', '17:00'),
  -- 10 Isabella (SJ): Tue-Thu 12-20
  (10, 'Tue', '12:00', '20:00'),
  (10, 'Wed', '12:00', '20:00'),
  (10, 'Thu', '12:00', '20:00'),
  -- 11 Ethan (ATX): nights
  (11, 'Mon', '18:00', '23:00'),
  (11, 'Tue', '18:00', '23:00'),
  (11, 'Fri', '18:00', '23:00'),
  -- 12 Harper (JAX): school hours
  (12, 'Mon', '09:00', '14:00'),
  (12, 'Wed', '09:00', '14:00'),
  (12, 'Fri', '09:00', '14:00'),
  -- 13 Logan (FTW): Thu-Sat
  (13, 'Thu', '10:00', '18:00'),
  (13, 'Fri', '10:00', '18:00'),
  (13, 'Sat', '10:00', '18:00'),
  -- 14 Amelia (CMH): Mon-Fri 08-16
  (14, 'Mon', '08:00', '16:00'),
  (14, 'Tue', '08:00', '16:00'),
  (14, 'Thu', '08:00', '16:00'),
  -- 15 Charlotte (CLT): live-in style
  (15, 'Tue', '07:00', '19:00'),
  (15, 'Thu', '07:00', '19:00'),
  (15, 'Sat', '08:00', '16:00'),
  -- 16 Jackson (SF): split shifts
  (16, 'Mon', '07:00', '11:00'),
  (16, 'Mon', '17:00', '21:00'),
  (16, 'Sat', '12:00', '18:00'),
  -- 17 Avery (SEA): Tue/Thu + Sun
  (17, 'Tue', '10:00', '18:00'),
  (17, 'Thu', '10:00', '18:00'),
  (17, 'Sun', '11:00', '17:00'),
  -- 18 William (DEN): Mon-Wed 06-14
  (18, 'Mon', '06:00', '14:00'),
  (18, 'Tue', '06:00', '14:00'),
  (18, 'Wed', '06:00', '14:00'),
  -- 19 Natalie (BOS): Tue-Fri 09-17
  (19, 'Tue', '09:00', '17:00'),
  (19, 'Wed', '09:00', '17:00'),
  (19, 'Fri', '09:00', '17:00'),
  -- 20 Maya (ATL): after-school + Sat
  (20, 'Mon', '15:00', '20:00'),
  (20, 'Wed', '15:00', '20:00'),
  (20, 'Sat', '09:00', '15:00');

  -- Seed languages

  INSERT INTO languages (caregiver_id, language) VALUES
  (1, 'English'),      (1, 'Spanish'),
  (2, 'English'),      (2, 'Spanish'),
  (3, 'English'),
  (4, 'English'),
  (5, 'English'),      (5, 'Spanish'),
  (6, 'English'),
  (7, 'English'),      (7, 'Vietnamese'),
  (8, 'English'),      (8, 'Korean'),
  (9, 'English'),
  (10,'English'),      (10,'Spanish'),
  (11,'English'),      (11,'Hindi'),
  (12,'English'),
  (13,'English'),
  (14,'English'),      (14,'French'),
  (15,'English'),
  (16,'English'),      (16,'Tagalog'),
  (17,'English'),      (17,'Mandarin'),
  (18,'English'),
  (19,'English'),      (19,'Portuguese'),
  (20,'English'),      (20,'ASL');

  -- Seed certifications

  INSERT INTO certifications (caregiver_id, cert) VALUES
  (1, 'CPR'),                 (1, 'First Aid'),
  (2, 'CPR'),                 (2, 'BLS'),
  (3, 'CPR'),
  (4, 'First Aid'),           (4, 'CNA'),
  (5, 'CPR'),                 (5, 'Dementia Care'),
  (6, 'HHA'),                 (6, 'CPR'),
  (7, 'First Aid'),
  (8, 'CPR'),                 (8, 'AED'),
  (9, 'CNA'),                 (9, 'BLS'),
  (10,'First Aid'),
  (11,'CPR'),                 (11,'Medication Technician'),
  (12,'CPR'),
  (13,'First Aid'),
  (14,'CPR'),                 (14,'CNA'),
  (15,'Hospice Training'),    (15,'CPR'),
  (16,'First Aid'),
  (17,'CPR'),                 (17,'Special Needs Training'),
  (18,'BLS'),                 (18,'CPR'),
  (19,'Dementia Care'),       (19,'First Aid'),
  (20,'CPR'),                 (20,'Childcare Safety');

  -- Seed traits

  INSERT INTO traits (caregiver_id, trait) VALUES
  (1, 'empathetic'),          (1, 'detail-oriented'),
  (2, 'patient'),             (2, 'communicative'),
  (3, 'cheerful'),            (3, 'reliable'),
  (4, 'organized'),           (4, 'professional'),
  (5, 'compassionate'),       (5, 'trustworthy'),
  (6, 'calm under pressure'), (6, 'punctual'),
  (7, 'adaptable'),           (7, 'energetic'),
  (8, 'friendly'),            (8, 'respectful'),
  (9, 'thorough'),            (9, 'dependable'),
  (10,'gentle'),              (10,'attentive'),
  (11,'proactive'),           (11,'communicative'),
  (12,'patient'),             (12,'kind'),
  (13,'flexible'),            (13,'tidy'),
  (14,'reliable'),            (14,'punctual'),
  (15,'compassionate'),       (15,'detail-oriented'),
  (16,'discreet'),            (16,'organized'),
  (17,'empathetic'),          (17,'calm under pressure'),
  (18,'professional'),        (18,'thorough'),
  (19,'trustworthy'),         (19,'proactive'),
  (20,'cheerful'),            (20,'attentive');

'''

# Insert the data into the database

sql = dedent(SQL)
con = sqlite3.connect(DB_PATH)
con.executescript(sql)
con.commit()
con.close()
print("Seeded:", DB_PATH)


Seeded: care_app.db


## Step 4: Teaching the AI to Search

Instead of the AI just talking, we teach it to actually *search* the database.

When someone says *"I need elder care in Boston for Mondays"*, the AI translates that into:
```python
search_caregivers(city="Boston", care_type="elder care", day="Mon")
```

This schema defines all the filters the AI can use: location, services, budget, language, availability, and more.

**This was the breakthrough.** No more writing custom queries—the AI figures it out.


In [66]:
# Tool definition schema

tools = [{
    "type": "function",
    "function": {
        "name": "search_caregivers",
        "description": (
            "Flexible multi-filter caregiver search. Any filter can be omitted. "
            "Supports location, service type, experience, pricing, live-in, language, "
            "certifications, day/time availability, and pagination."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "City name to filter by (optional)."
                },
                "state_province": {
                    "type": "string",
                    "description": "State or province to filter by (optional)."
                },
                "country": {
                    "type": "string",
                    "description": "Country to filter by (optional)."
                },
                "care_type": {
                    "type": "string",
                    "description": (
                        "Service category, e.g., 'elder_care', 'child_care', "
                        "'pet_care', 'housekeeping' (optional)."
                    )
                },
                "min_experience": {
                    "type": "integer",
                    "minimum": 0,
                    "description": "Minimum years of experience (optional)."
                },
                "max_hourly_rate": {
                    "type": "number",
                    "minimum": 0,
                    "description": "Maximum hourly rate in local currency (optional)."
                },
                "live_in": {
                    "type": "boolean",
                    "description": "Require live-in caregivers (optional)."
                },
                "language": {
                    "type": "string",
                    "description": "Required spoken language, e.g., 'English', 'Spanish' (optional)."
                },
                "certification": {
                    "type": "string",
                    "description": "Required certification, e.g., 'CPR', 'CNA' (optional)."
                },
                "day": {
                    "type": "string",
                    "description": (
                        "Day of week to match availability (optional), e.g., "
                        "'Monday', 'Tuesday', ... 'Sunday'."
                    )
                },
                "time_between": {
                    "type": "array",
                    "description": (
                        "Required availability window as ['HH:MM','HH:MM'] in 24h time. "
                        "Matches caregivers whose availability window fully covers this range."
                    ),
                    "items": {
                        "type": "string",
                        "pattern": "^\\d{2}:\\d{2}$",
                        "description": "Time in 'HH:MM' 24-hour format."
                    },
                    "minItems": 2,
                    "maxItems": 2
                },
                "limit": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 1000,
                    "default": 50,
                    "description": "Max number of results to return (default 50)."
                },
                "offset": {
                    "type": "integer",
                    "minimum": 0,
                    "default": 0,
                    "description": "Number of results to skip for pagination (default 0)."
                }
            },
            "required": [],
            "additionalProperties": False
        }
    }
}]

tools

[{'type': 'function',
  'function': {'name': 'search_caregivers',
   'description': 'Flexible multi-filter caregiver search. Any filter can be omitted. Supports location, service type, experience, pricing, live-in, language, certifications, day/time availability, and pagination.',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string',
      'description': 'City name to filter by (optional).'},
     'state_province': {'type': 'string',
      'description': 'State or province to filter by (optional).'},
     'country': {'type': 'string',
      'description': 'Country to filter by (optional).'},
     'care_type': {'type': 'string',
      'description': "Service category, e.g., 'elder_care', 'child_care', 'pet_care', 'housekeeping' (optional)."},
     'min_experience': {'type': 'integer',
      'minimum': 0,
      'description': 'Minimum years of experience (optional).'},
     'max_hourly_rate': {'type': 'number',
      'minimum': 0,
      'description': 'Maximum

## Step 5: Helper Functions

**Voice:** The AI can speak its responses using OpenAI's text-to-speech.

**Database functions:** All the queries we need—search, get profiles, check availability, etc. These are what the AI calls behind the scenes.


In [67]:

# Convert text to speech using OpenAI's TTS API
def announcements(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="coral",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

# Context manager for database connection
@contextmanager
def _conn(dict_rows: bool = True):
    conn = sqlite3.connect(DB_PATH)
    if dict_rows:
        conn.row_factory = _dict_factory
    try:
        yield conn
        conn.commit()
    finally:
        conn.close()

####################
# Helper functions #
####################

# Converts SQLite query results from tuples into dictionaries
def _dict_factory(cursor, row):
    return {col[0]: row[idx] for idx, col in enumerate(cursor.description)}
# A debug/logging function that prints database tool activity
def _print(msg: str):
    print(f"DATABASE TOOL CALLED: {msg}", flush=True)

################################
# Caregiver database functions #
################################

# Counts the number of caregivers in the database
def get_caregiver_count() -> int:
    _print("Counting caregivers")
    with _conn() as conn:
        cur = conn.cursor()
        cur.execute("SELECT COUNT(*) AS n FROM caregivers")
        return cur.fetchone()["n"]

# Fetches a caregiver's profile by their ID
def get_caregiver(caregiver_id: int) -> Optional[Dict[str, Any]]:
    _print(f"Fetching caregiver #{caregiver_id}")
    with _conn() as conn:
        cur = conn.cursor()
        cur.execute("SELECT * FROM caregivers WHERE id = ?", (caregiver_id,))
        return cur.fetchone()

# Lists caregivers with pagination
def list_caregivers(limit: int = 20, offset: int = 0) -> List[Dict[str, Any]]:
    _print(f"Listing caregivers (limit={limit}, offset={offset})")
    with _conn() as conn:
        cur = conn.cursor()
        cur.execute("""
            SELECT * FROM caregivers
            ORDER BY id
            LIMIT ? OFFSET ?
        """, (limit, offset))
        return cur.fetchall()

# Fetches the services a caregiver offers
def get_services(caregiver_id: int) -> List[str]:
    _print(f"Fetching services for caregiver #{caregiver_id}")
    with _conn() as conn:
        cur = conn.cursor()
        cur.execute("""
            SELECT care_type FROM caregiver_services WHERE caregiver_id = ?
            ORDER BY care_type
        """, (caregiver_id,))
        return [r["care_type"] for r in cur.fetchall()]

# Fetches the languages a caregiver speaks
def get_languages(caregiver_id: int) -> List[str]:
    _print(f"Fetching languages for caregiver #{caregiver_id}")
    with _conn() as conn:
        cur = conn.cursor()
        cur.execute("""
            SELECT language FROM languages WHERE caregiver_id = ?
            ORDER BY language
        """, (caregiver_id,))
        return [r["language"] for r in cur.fetchall()]

# Fetches the certifications a caregiver has
def get_certifications(caregiver_id: int) -> List[str]:
    _print(f"Fetching certifications for caregiver #{caregiver_id}")
    with _conn() as conn:
        cur = conn.cursor()
        cur.execute("""
            SELECT cert FROM certifications WHERE caregiver_id = ?
            ORDER BY cert
        """, (caregiver_id,))
        return [r["cert"] for r in cur.fetchall()]

# Fetches the traits a caregiver has
def get_traits(caregiver_id: int) -> List[str]:
    _print(f"Fetching traits for caregiver #{caregiver_id}")
    with _conn() as conn:
        cur = conn.cursor()
        cur.execute("""
            SELECT trait FROM traits WHERE caregiver_id = ?
            ORDER BY trait
        """, (caregiver_id,))
        return [r["trait"] for r in cur.fetchall()]

# Fetches the availability of a caregiver
def get_availability(caregiver_id: int) -> List[Dict[str, str]]:
    _print(f"Fetching availability for caregiver #{caregiver_id}")
    with _conn() as conn:
        cur = conn.cursor()
        cur.execute("""
            SELECT day, time_start, time_end
            FROM availability
            WHERE caregiver_id = ?
            ORDER BY
              CASE day
                WHEN 'Mon' THEN 1 WHEN 'Tue' THEN 2 WHEN 'Wed' THEN 3
                WHEN 'Thu' THEN 4 WHEN 'Fri' THEN 5 WHEN 'Sat' THEN 6
                WHEN 'Sun' THEN 7 ELSE 8
              END, time_start
        """, (caregiver_id,))
        return cur.fetchall()

# Fetches a caregiver's full profile
def get_caregiver_profile(caregiver_id: int) -> Optional[Dict[str, Any]]:
    _print(f"Fetching full profile for caregiver #{caregiver_id}")
    base = get_caregiver(caregiver_id)
    if not base:
        return None
    base["services"] = get_services(caregiver_id)
    base["languages"] = get_languages(caregiver_id)
    base["certifications"] = get_certifications(caregiver_id)
    base["traits"] = get_traits(caregiver_id)
    base["availability"] = get_availability(caregiver_id)
    return base

###########################################
# Search caregivers with multiple filters #
###########################################

def search_caregivers(
    city: Optional[str] = None,
    state_province: Optional[str] = None,
    country: Optional[str] = None,
    care_type: Optional[str] = None,
    min_experience: Optional[int] = None,
    max_hourly_rate: Optional[float] = None,
    live_in: Optional[bool] = None,
    language: Optional[str] = None,
    certification: Optional[str] = None,
    day: Optional[str] = None,
    time_between: Optional[Tuple[str, str]] = None,  # ('HH:MM', 'HH:MM')
    limit: int = 50,
    offset: int = 0
) -> List[Dict[str, Any]]:
    """
    Flexible multi-filter search. Any filter can be omitted.
    """
    _print("Searching caregivers with multiple filters")

    # base + optional joins
    join_clauses = []
    where = ["1=1"]
    params: List[Any] = []

    if care_type:
        join_clauses.append("JOIN caregiver_services s ON s.caregiver_id = c.id")
        where.append("LOWER(s.care_type) = LOWER(?)")
        params.append(care_type)

    if language:
        join_clauses.append("JOIN languages l ON l.caregiver_id = c.id")
        where.append("LOWER(l.language) = LOWER(?)")
        params.append(language)

    if certification:
        join_clauses.append("JOIN certifications cert ON cert.caregiver_id = c.id")
        where.append("LOWER(cert.cert) = LOWER(?)")
        params.append(certification)

    if day or time_between:
        join_clauses.append("JOIN availability a ON a.caregiver_id = c.id")
        if day:
            where.append("a.day = ?")
            params.append(day)
        if time_between:
            t0, t1 = time_between
            # overlap check: caregiver window [start,end] must include [t0,t1]
            where.append("a.time_start <= ? AND a.time_end >= ?")
            params.extend([t0, t1])

    if city:
        where.append("LOWER(c.city) = LOWER(?)")
        params.append(city)
    if state_province:
        where.append("LOWER(c.state_province) = LOWER(?)")
        params.append(state_province)
    if country:
        where.append("LOWER(c.country) = LOWER(?)")
        params.append(country)
    if min_experience is not None:
        where.append("c.years_experience >= ?")
        params.append(min_experience)
    if max_hourly_rate is not None:
        where.append("c.hourly_rate <= ?")
        params.append(max_hourly_rate)
    if live_in is not None:
        where.append("c.live_in = ?")
        params.append(1 if live_in else 0)

    sql = f"""
        SELECT DISTINCT c.*
        FROM caregivers c
        {' '.join(join_clauses)}
        WHERE {' AND '.join(where)}
        ORDER BY c.hourly_rate ASC, c.years_experience DESC, c.id
        LIMIT ? OFFSET ?
    """
    params.extend([limit, offset])

    with _conn() as conn:
        cur = conn.cursor()
        cur.execute(sql, tuple(params))
        return cur.fetchall()

## Step 6: Quick Test

Before connecting everything to the AI, let's make sure the database works. Run these examples to see sample caregivers and their profiles.


In [68]:
# Example 1: Search for elder care providers in New York
results = search_caregivers(
    city="New York",
    care_type="elder care",
    max_hourly_rate=30.0,
    limit=5
)

print(f"Found {len(results)} elder care providers in New York:")
for caregiver in results:
    print(f"- {caregiver['name']}: ${caregiver['hourly_rate']}/hr, {caregiver['years_experience']} years experience")

print("\n" + "="*60 + "\n")

# Example 2: Search for Spanish-speaking child care providers
results2 = search_caregivers(
    care_type="child care",
    language="Spanish",
    limit=3
)

print(f"Found {len(results2)} Spanish-speaking child care providers:")
for caregiver in results2:
    print(f"- {caregiver['name']} in {caregiver['city']}, {caregiver['state_province']}")

print("\n" + "="*60 + "\n")

# Example 3: Get detailed profile of a specific caregiver
if results:
    caregiver_id = results[0]['id']
    profile = get_caregiver_profile(caregiver_id)
    print(f"Detailed profile for {profile['name']}:")
    print(f"  Services: {', '.join(profile['services'])}")
    print(f"  Languages: {', '.join(profile['languages'])}")
    print(f"  Certifications: {', '.join(profile['certifications'])}")
    print(f"  Traits: {', '.join(profile['traits'])}")
    print(f"  Availability: {len(profile['availability'])} time slots")


DATABASE TOOL CALLED: Searching caregivers with multiple filters
Found 1 elder care providers in New York:
- Grace Williams: $28.0/hr, 6 years experience


DATABASE TOOL CALLED: Searching caregivers with multiple filters
Found 1 Spanish-speaking child care providers:
- Isabella Garcia in San Jose, CA


DATABASE TOOL CALLED: Fetching full profile for caregiver #1
DATABASE TOOL CALLED: Fetching caregiver #1
DATABASE TOOL CALLED: Fetching services for caregiver #1
DATABASE TOOL CALLED: Fetching languages for caregiver #1
DATABASE TOOL CALLED: Fetching certifications for caregiver #1
DATABASE TOOL CALLED: Fetching traits for caregiver #1
DATABASE TOOL CALLED: Fetching availability for caregiver #1
Detailed profile for Grace Williams:
  Services: companionship, elder care
  Languages: English, Spanish
  Certifications: CPR, First Aid
  Traits: detail-oriented, empathetic
  Availability: 3 time slots


## Step 7: The AI's Instructions

Here's where I learned prompt engineering matters *a lot*.

The AI needs to know:
- What exact keywords to use ("elder care" not "elderly care", "Mon" not "Monday")
- How to map natural language to database values
- That it should give 2-3 recommendations with pros/cons
- To remind families to verify credentials independently

**The lesson from MyWoosah:** Small keyword mismatches = zero results. This prompt prevents that.


In [69]:
# System prompt

system_prompt = '''
    You are a compassionate Caregiver Assistant that helps families quickly identify the most
    suitable care provider by gathering requirements (care needs, schedule, budget, location,
    language/cultural fit) and matching them to available profiles. Provide 2-3 best-fit options
    with pros/cons, estimated costs, and next steps, and clearly state that credentials/background
    checks are not verified by this sample app and should be confirmed by the family.

    CRITICAL: When searching the database, you MUST use these EXACT terms:

    CARE TYPES (use exactly as shown):
    - "elder care" (for elderly, senior, old age, geriatric care)
    - "companionship" (for companion, friendship, social support)
    - "post-op support" (for post-surgery, post-operative, recovery care)
    - "child care" (for children, kids, babysitting, nanny)
    - "special needs" (for disabilities, autism, developmental needs)
    - "respite care" (for temporary relief, break for family caregivers)
    - "dementia care" (for Alzheimer's, memory care, cognitive decline)
    - "hospice support" (for end-of-life, palliative, terminal care)

    If a user mentions any variation, map it to the closest match above. If unclear, ask clarifying questions.

    DAYS OF WEEK (use exactly as shown):
    - "Mon" (for Monday)
    - "Tue" (for Tuesday)
    - "Wed" (for Wednesday)
    - "Thu" (for Thursday)
    - "Fri" (for Friday)
    - "Sat" (for Saturday)
    - "Sun" (for Sunday)

    STATES/PROVINCES (use 2-letter codes):
    - Use standard US state abbreviations: "NY", "CA", "TX", "FL", "MA", etc.
    - Convert full state names to abbreviations before searching

    COMMON LANGUAGES:
    - "English", "Spanish", "French", "Vietnamese", "Korean", "Hindi", "Mandarin", "Portuguese", "Tagalog", "ASL"
    - Capitalize properly (e.g., user says "spanish" → use "Spanish")

    CERTIFICATIONS:
    - "CPR", "First Aid", "CNA", "BLS", "HHA", "AED", "Medication Technician", "Hospice Training", 
    "Dementia Care", "Special Needs Training", "Childcare Safety"
    - Use exact capitalization and full names

    TRAITS:
    - "empathetic", "patient", "cheerful", "organized", "compassionate", "calm under pressure", 
    "adaptable", "friendly", "thorough", "gentle", "proactive", "flexible", "reliable", 
    "detail-oriented", "communicative", "energetic", "respectful", "dependable", "attentive", 
    "kind", "tidy", "punctual", "discreet", "professional", "trustworthy"
    - Use lowercase for all traits

    SEARCH STRATEGY:
    1. Listen carefully to user requirements
    2. Map their natural language to database terms above
    3. Use search_caregivers() with exact keyword matches
    4. If no results, suggest alternatives or broader searches
    5. After getting results, use get_caregiver_profile() for detailed information on top matches

    Always confirm your understanding by restating requirements using the exact database terms before searching.
'''

## Step 8: Making it Work (and Not Crash)

This is the engine room. When the AI wants to search, this code:
1. Validates the request
2. Calls the right database function
3. Handles errors gracefully (no crashes!)
4. Limits results to prevent overwhelming the AI
5. Generates the voice response

**Defensive programming:** I learned the hard way that things break. This code expects problems and handles them elegantly.


In [70]:
# Function registry: Maps tool names to actual Python functions
TOOL_REGISTRY = {
    "search_caregivers": search_caregivers,
    "get_caregiver_count": get_caregiver_count,
    "get_caregiver": get_caregiver,
    "list_caregivers": list_caregivers,
    "get_services": get_services,
    "get_languages": get_languages,
    "get_certifications": get_certifications,
    "get_traits": get_traits,
    "get_availability": get_availability,
    "get_caregiver_profile": get_caregiver_profile,
}

def execute_tool_call(tool_call):
    """
    Safely execute a single tool call with error handling.
    Returns a properly formatted tool response.
    """
    import json
    
    function_name = tool_call.function.name
    
    # Defensive check: Ensure function exists in registry
    if function_name not in TOOL_REGISTRY:
        return {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps({
                "error": f"Unknown function: {function_name}",
                "available_functions": list(TOOL_REGISTRY.keys())
            })
        }
    
    try:
        # Parse arguments
        args = json.loads(tool_call.function.arguments)
        
        # Execute the function
        func = TOOL_REGISTRY[function_name]
        result = func(**args)
        
        # Format response based on result type with limit to prevent token overflow
        if isinstance(result, list):
            content = json.dumps({
                "count": len(result),
                "results": result[:10] if len(result) > 10 else result,
                "truncated": len(result) > 10
            })
        elif isinstance(result, dict):
            content = json.dumps(result)
        elif isinstance(result, (int, float, str)):
            content = json.dumps({"result": result})
        else:
            content = str(result)
            
        return {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": content
        }
        
    except Exception as e:
        # Defensive error handling
        return {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps({
                "error": str(e),
                "function": function_name,
                "args": tool_call.function.arguments
            })
        }

def process_tool_calls(message):
    """
    Process all tool calls from the AI response.
    Returns tool responses and extracted metadata.
    """
    responses = []
    metadata = {
        "cities": set(),
        "caregiver_ids": set(),
        "total_results": 0
    }
    
    if not message.tool_calls:
        return responses, metadata
    
    for tool_call in message.tool_calls:
        # Execute the tool call
        response = execute_tool_call(tool_call)
        responses.append(response)
        
        # Extract metadata for UI enhancements
        try:
            import json
            content = json.loads(response["content"])
            
            # Extract cities from search results
            if "results" in content and isinstance(content["results"], list):
                for item in content["results"]:
                    if isinstance(item, dict) and "city" in item:
                        metadata["cities"].add(item["city"])
                    if isinstance(item, dict) and "id" in item:
                        metadata["caregiver_ids"].add(item["id"])
                        
            if "count" in content:
                metadata["total_results"] += content["count"]
                
        except:
            pass  # Silently ignore metadata extraction errors
    
    return responses, metadata

def generate_city_image(city):
    """
    Generate or retrieve a city image (placeholder for future enhancement).
    Could integrate with DALL-E, Unsplash API, or local image database.
    """
    # Placeholder - can be enhanced with actual image generation
    return None

def chat(history):
    """
    Main chat handler with multi-modal support and defensive error handling.
    Handles conversation flow, tool calls, and response generation.
    """
    # Normalize history format
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Initialize conversation with system prompt
    messages = [{"role": "system", "content": system_prompt}] + history
    
    # Initialize metadata
    image = None
    selected_city = None
    
    try:
        # Initial API call
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
        
        # Tool calling loop (with safety limit)
        max_iterations = 5
        iteration = 0
        
        while response.choices[0].finish_reason == "tool_calls" and iteration < max_iterations:
            iteration += 1
            message = response.choices[0].message
            
            # Process all tool calls
            tool_responses, metadata = process_tool_calls(message)
            
            # Track city for image generation
            if metadata["cities"]:
                selected_city = list(metadata["cities"])[0]
            
            # Add assistant message and tool responses to conversation
            messages.append(message)
            messages.extend(tool_responses)
            
            # Continue conversation
            response = openai.chat.completions.create(
                model=MODEL,
                messages=messages,
                tools=tools
            )
        
        # Extract final reply
        reply = response.choices[0].message.content
        history.append({"role": "assistant", "content": reply})
        
        # Generate voice response
        voice = announcements(reply)
        
        # Generate city image if applicable
        if selected_city:
            image = generate_city_image(selected_city)
        
        return history, voice, image
        
    except Exception as e:
        # Defensive error handling for the entire chat flow
        error_message = f"I apologize, but I encountered an error: {str(e)}. Please try again."
        history.append({"role": "assistant", "content": error_message})
        return history, None, None

## Step 9: The Interface

A clean, professional web UI built with Gradio.

Features:
- Chat interface with conversation history
- Voice responses that auto-play
- Settings sidebar (model selection, voice options)
- Clear instructions for families

**Why Gradio?** At MyWoosah, I needed something non-technical staff could use immediately. Gradio made that possible without weeks of frontend work.

**Run this cell to launch!** 🚀


In [71]:
import gradio as gr

# Gradio UI Setup

def put_message_in_chatbot(message, history):
    """Add user message to chat history"""
    return "", history + [{"role": "user", "content": message}]

# Custom CSS for better styling
custom_css = """
#chatbot {
    border-radius: 10px;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
}
#message_box {
    border-radius: 8px;
}
.header {
    text-align: center;
    padding: 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 10px;
    margin-bottom: 20px;
}
"""

with gr.Blocks(title="CareGiver AI Assistant", css=custom_css, theme=gr.themes.Soft()) as ui:
    
    # Header
    gr.Markdown("""
    <div class="header">
        <h1>🏥 RoboCare AI Assistant</h1>
        <p>Find the perfect caregiver for your loved ones</p>
    </div>
    """)
    
    # Instructions
    with gr.Accordion("ℹ️ Click here to learn more on how to use this AI", open=False):
        gr.Markdown("""
        **Tell me what you need:**
        - Type of care (elder care, child care, companionship, etc.)
        - Location (city, state)
        - Schedule requirements (days/times)
        - Budget constraints
        - Language or certification needs
        
        **Example:** "I need an elder care provider in Boston for Monday mornings who speaks Spanish and has CPR certification."
        
        ⚠️ **Note:** This is a demo app. Always verify credentials and conduct background checks independently.
        """)
    
    # Main chat interface
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                height=500, 
                type="messages",
                elem_id="chatbot",
                label="Chat History",
                avatar_images=(None, "🤖")
            )
            
            # Audio output
            audio_output = gr.Audio(
                label="Voice Response",
                autoplay=True,
                visible=True,
                interactive=False
            )
        
        # Settings sidebar
        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Settings")
            
            # Model selector (for future enhancement)
            model_select = gr.Dropdown(
                choices=["gpt-4o-mini", "gpt-4o", "gpt-4-turbo"],
                value="gpt-4o-mini",
                label="AI Model",
                interactive=True
            )
            
            # Voice selector
            voice_select = gr.Dropdown(
                choices=["coral", "alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                value="coral",
                label="Voice",
                interactive=True
            )
            
            # Audio toggle
            audio_enabled = gr.Checkbox(
                label="Enable Voice Responses",
                value=True
            )
            
            # Clear button
            clear_btn = gr.Button("🗑️ Clear Conversation", variant="secondary")
    
    # Input section
    with gr.Row():
        message = gr.Textbox(
            label="Your Message",
            placeholder="Type your question here... (e.g., 'I need elder care in Boston')",
            lines=2,
            elem_id="message_box",
            scale=4
        )
        send_btn = gr.Button("Send", variant="primary", scale=1)
    
    # Event handlers
    def chat_wrapper(history):
        """Wrapper to handle chat and extract only needed outputs"""
        history_out, voice, image = chat(history)
        return history_out, voice
    
    # Submit on enter or button click
    submit_event = message.submit(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat_wrapper,
        inputs=chatbot,
        outputs=[chatbot, audio_output]
    )
    
    send_btn.click(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat_wrapper,
        inputs=chatbot,
        outputs=[chatbot, audio_output]
    )
    
    # Clear conversation
    clear_btn.click(
        lambda: ([], None),
        outputs=[chatbot, audio_output]
    )
    
    # Footer
    gr.Markdown("""
    ---
    <center>
    <small>Powered by OpenAI & Gradio | Built by RoboOffice Ltd</small>
    </center>
    """)

# Launch with better configuration
ui.launch(
    inbrowser=True,
    share=False,
    show_error=True,
    quiet=False
)

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


---

## Reflections

This project started from frustration: *"There has to be a better way to match families with caregivers."*

Through the Andela program, I learned that AI + thoughtful engineering = solutions to real problems.

### What Worked:
- **Function calling** eliminated the need for custom queries
- **Prompt engineering** prevented keyword mismatches
- **Defensive coding** made it robust
- **Gradio** made it accessible

### What I'd Do Next:
- Add speech input (families could call and talk)
- Connect to actual MyWoosah database
- Add background check API integration
- Deploy for real users

### The Bigger Picture:

This isn't just about caregiving. The same pattern works for:
- Healthcare appointments
- Legal services
- Tutoring platforms
- Any matching problem where natural language beats forms

AI doesn't replace good database design—it makes it accessible to everyone.

---

**For MyWoosah Inc and beyond:** This is proof that AI can transform how we connect people with the care they need.

*Built during Week 2 of the Andela LLM Engineering Program*
